In [1]:
import random
import csv
import time, array
import numpy as np
import autograd.numpy as anp
import pandas as pd

from math import sin, cos, pi, exp, e, sqrt, factorial

from deap import base, creator, tools, algorithms, benchmarks 
from deap.benchmarks.tools import diversity, convergence, hypervolume

from scipy.spatial.distance import cdist

from operator import mul
from functools import reduce

import pymop

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

In [2]:
def mutInversion(individual):
    size = len(individual)
    if size == 0:
        return individual,

    index_one = random.randrange(size)
    index_two = random.randrange(size)
    start_index = min(index_one, index_two)
    end_index = max(index_one, index_two)

    # Reverse the contents of the individual between the indices
    individual[start_index:end_index] = individual[start_index:end_index][::-1]

    return individual,

In [3]:
def measure_time(x):
    execution_time = time.time() - start_time 
#     print(execution_time)
    return execution_time

In [40]:
# def nsga_ii(toolbox, seed, stats=None, verbose=False):
#     pop = toolbox.population(n=toolbox.pop_size)
#     pop = toolbox.select(pop, len(pop))
#     random.seed(seed)
#     return algorithms.eaMuPlusLambda(pop, 
#                                      toolbox, 
#                                      mu=toolbox.pop_size, 
#                                      lambda_=toolbox.pop_size, 
#                                      cxpb=toolbox.cross_prob,
#                                      mutpb=toolbox.mut_prob,
#                                      ngen=toolbox.max_gen,
#                                      stats=stats, 
#                                      verbose=verbose)

In [4]:
def nsga_ii(toolbox, seed, stats=None, verbose=False):
    random.seed(seed)
    pop = toolbox.population(n=toolbox.pop_size)
    fitnesses = toolbox.map(toolbox.evaluate, pop)
    
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    pop = toolbox.select(pop, len(pop))
    
    for gen in range(1, NGEN):
        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]

        for ind1, ind2 in zip(offspring[::2],offspring[1::2]):
            if random.random() <= toolbox.cross_prob:
                toolbox.map(ind1, ind2)
                
            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            
            del ind1.fitness.values, ind2.fitness.values
        
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
               
        record = stats.compile(pop) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)
        
        pop = toolbox.select(pop+offspring,toolbox.pop_size)
    
    return pop, logbook

In [5]:
def nsga_iii(toolbox,seed,stats,logbook,NGEN,MU,CXPB,MUTPB,verbose=False):
    random.seed(seed)
    pop = toolbox.population(n=MU)
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
#   fitnesses = toolbox.map(toolbox.evaluate, pop)        
#     for ind, fit in zip(pop, fitnesses):
#         ind.fitness.values = fit
    
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop = toolbox.select(pop, len(pop))
    
    # Begin the generational process
    for gen in range(1, NGEN):
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        record = stats.compile(pop) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        
        if verbose:
            print(logbook.stream)
        
        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
    
    return pop, logbook

In [6]:
kroA100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroA100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroB100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroB100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroC100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroC100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")

In [7]:
one = cdist(kroA100.values,kroA100.values)
two = cdist(kroB100.values,kroB100.values)
three = cdist(kroC100.values,kroC100.values)

In [8]:
def motsp(x, one=one, two=two, three=three):
        n_cities = len(x)
        dist_one = 0
        dist_two = 0
        dist_three = 0
        
        for k in range(n_cities - 1):
            i, j = x[k], x[k + 1]
            dist_one += one[i, j]
            dist_two += two[i, j]
            dist_three += three[i, j]

        last, first = x[-1], x[0]
        
        dist_one += one[last, first]
        dist_one = 100000/dist_one
        dist_two += two[last, first]
        dist_two = 100000/dist_two
        dist_three += three[last, first]
        dist_three = 100000/dist_three
        
        return dist_one, dist_two, dist_three

In [10]:
NOBJ = 3
NGEN = 5000
CXPB = 0.8
MUTPB = 0.2

P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
ref_points = tools.uniform_reference_points(NOBJ, P)
MU = 92

IND_SIZE=100

In [11]:
for index in range(1, 11):  # Adjust the range as needed
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ) 
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("indices", random.sample, range(100), IND_SIZE)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", motsp) #Fitness Function

    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", mutInversion)
    toolbox.register("select", tools.selNSGA2)

    toolbox.pop_size = MU
    toolbox.max_gen = NGEN
    toolbox.mut_prob = MUTPB
    toolbox.cross_prob = CXPB

    # Initialize statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("Tiempo", measure_time)

    logbook = tools.Logbook()
    
    logbook.header = "gen", "evals", "Tiempo"
    
    print(f"Iteration {index}")
    start_time = time.time()
    res, logbook = nsga_ii(toolbox=toolbox, seed=index, stats=stats, verbose=True)

    fronts = tools.emo.sortLogNondominated(res, len(res))

    for i, inds in enumerate(fronts):
        par = [toolbox.evaluate(ind) for ind in inds]
        df = pd.DataFrame(par)

    df.to_csv(f'deap_fitness_nsga2_motsp_92_500_3_100-{index}.csv', index=False)
    log = pd.DataFrame(logbook)
    log.to_csv(f'deap_time_nsga2_motsp_92_500_3_100-{index}.csv', index=False)
    del(log, df, par, fronts, logbook, res, start_time)

Iteration 1
gen	evals	Tiempo   
1  	     	0.0689228
2  	     	0.125094 
3  	     	0.180571 
4  	     	0.235876 
5  	     	0.30447  
6  	     	0.353186 
7  	     	0.394885 
8  	     	0.436144 
9  	     	0.485083 
10 	     	0.542797 
11 	     	0.597006 
12 	     	0.646188 
13 	     	0.702804 
14 	     	0.755091 
15 	     	0.821373 
16 	     	0.880146 
17 	     	0.930581 
18 	     	0.976393 
19 	     	1.03029  
20 	     	1.07597  
21 	     	1.14371  
22 	     	1.19046  
23 	     	1.2392   
24 	     	1.28502  
25 	     	1.33898  
26 	     	1.39218  
27 	     	1.44933  
28 	     	1.50892  
29 	     	1.56911  
30 	     	1.60843  
31 	     	1.65588  
32 	     	1.70971  
33 	     	1.76103  
34 	     	1.82548  
35 	     	1.86914  
36 	     	1.91529  
37 	     	1.97484  
38 	     	2.0241   
39 	     	2.09546  
40 	     	2.16993  
41 	     	2.23186  
42 	     	2.29628  
43 	     	2.36484  
44 	     	2.42812  
45 	     	2.49602  
46 	     	2.54924  
47 	     	2.59857  
48 	     	2.66486  
49 	    

412	     	23.0364  
413	     	23.0848  
414	     	23.1451  
415	     	23.194   
416	     	23.2671  
417	     	23.3154  
418	     	23.4166  
419	     	23.5318  
420	     	23.6654  
421	     	23.7369  
422	     	23.7766  
423	     	23.8364  
424	     	23.885   
425	     	23.9329  
426	     	23.9853  
427	     	24.0263  
428	     	24.0731  
429	     	24.125   
430	     	24.1829  
431	     	24.2263  
432	     	24.2798  
433	     	24.3337  
434	     	24.3929  
435	     	24.4452  
436	     	24.494   
437	     	24.5379  
438	     	24.5852  
439	     	24.6348  
440	     	24.6903  
441	     	24.7449  
442	     	24.8529  
443	     	24.9708  
444	     	25.096   
445	     	25.213   
446	     	25.3656  
447	     	25.4445  
448	     	25.5181  
449	     	25.6009  
450	     	25.6647  
451	     	25.7397  
452	     	25.809   
453	     	25.8689  
454	     	25.9397  
455	     	26.0192  
456	     	26.0985  
457	     	26.156   
458	     	26.2042  
459	     	26.2591  
460	     	26.3329  
461	     	26.3932  


823	     	48.2389  
824	     	48.3078  
825	     	48.3802  
826	     	48.4519  
827	     	48.5168  
828	     	48.5832  
829	     	48.6801  
830	     	48.7509  
831	     	48.8178  
832	     	48.9138  
833	     	49.0074  
834	     	49.0866  
835	     	49.1833  
836	     	49.2728  
837	     	49.3603  
838	     	49.4376  
839	     	49.5295  
840	     	49.6127  
841	     	49.6637  
842	     	49.7442  
843	     	49.8335  
844	     	49.9221  
845	     	49.9816  
846	     	50.0548  
847	     	50.1303  
848	     	50.2259  
849	     	50.3118  
850	     	50.4003  
851	     	50.4959  
852	     	50.5824  
853	     	50.6678  
854	     	50.767   
855	     	50.859   
856	     	50.941   
857	     	51.0257  
858	     	51.1218  
859	     	51.2289  
860	     	51.2873  
861	     	51.3472  
862	     	51.404   
863	     	51.4767  
864	     	51.5253  
865	     	51.5787  
866	     	51.6377  
867	     	51.6928  
868	     	51.7508  
869	     	51.8057  
870	     	51.8557  
871	     	51.917   
872	     	51.9698  


1222	     	69.3969  
1223	     	69.4508  
1224	     	69.501   
1225	     	69.5477  
1226	     	69.5986  
1227	     	69.6582  
1228	     	69.7348  
1229	     	69.7878  
1230	     	69.8307  
1231	     	69.8787  
1232	     	69.9197  
1233	     	69.9718  
1234	     	70.0176  
1235	     	70.0759  
1236	     	70.1258  
1237	     	70.1776  
1238	     	70.2187  
1239	     	70.265   
1240	     	70.311   
1241	     	70.3519  
1242	     	70.3992  
1243	     	70.4447  
1244	     	70.4839  
1245	     	70.532   
1246	     	70.5771  
1247	     	70.6172  
1248	     	70.6619  
1249	     	70.7198  
1250	     	70.7697  
1251	     	70.8149  
1252	     	70.8608  
1253	     	70.9037  
1254	     	70.9444  
1255	     	70.9886  
1256	     	71.0375  
1257	     	71.0791  
1258	     	71.1274  
1259	     	71.1817  
1260	     	71.2243  
1261	     	71.2697  
1262	     	71.3186  
1263	     	71.362   
1264	     	71.4076  
1265	     	71.459   
1266	     	71.5107  
1267	     	71.5526  
1268	     	71.6025  
1269	     	71

1616	     	88.8148  
1617	     	88.8648  
1618	     	88.9129  
1619	     	88.9529  
1620	     	89.0053  
1621	     	89.0452  
1622	     	89.0956  
1623	     	89.1449  
1624	     	89.196   
1625	     	89.2428  
1626	     	89.2902  
1627	     	89.3438  
1628	     	89.3902  
1629	     	89.4619  
1630	     	89.5241  
1631	     	89.5652  
1632	     	89.6153  
1633	     	89.6648  
1634	     	89.7062  
1635	     	89.7652  
1636	     	89.8155  
1637	     	89.8631  
1638	     	89.9073  
1639	     	89.9529  
1640	     	89.9961  
1641	     	90.0379  
1642	     	90.0947  
1643	     	90.1353  
1644	     	90.1927  
1645	     	90.2379  
1646	     	90.2828  
1647	     	90.3448  
1648	     	90.4597  
1649	     	90.5252  
1650	     	90.5939  
1651	     	90.6394  
1652	     	90.693   
1653	     	90.7458  
1654	     	90.793   
1655	     	90.8379  
1656	     	90.8946  
1657	     	90.9427  
1658	     	90.9926  
1659	     	91.0428  
1660	     	91.0912  
1661	     	91.145   
1662	     	91.1892  
1663	     	91

...

3195	     	168.913  
3196	     	169      
3197	     	169.072  
3198	     	169.147  
3199	     	169.212  
3200	     	169.275  
3201	     	169.369  
3202	     	169.461  
3203	     	169.537  
3204	     	169.617  
3205	     	169.703  
3206	     	169.781  
3207	     	169.903  
3208	     	170.033  
3209	     	170.193  
3210	     	170.357  
3211	     	170.516  
3212	     	170.642  
3213	     	170.737  
3214	     	170.79   
3215	     	170.866  
3216	     	170.957  
3217	     	171.066  
3218	     	171.173  
3219	     	171.266  
3220	     	171.357  
3221	     	171.462  
3222	     	171.586  
3223	     	171.697  
3224	     	171.786  
3225	     	171.891  
3226	     	172.017  
3227	     	172.123  
3228	     	172.223  
3229	     	172.319  
3230	     	172.387  
3231	     	172.443  
3232	     	172.499  
3233	     	172.559  
3234	     	172.62   
3235	     	172.685  
3236	     	172.739  
3237	     	172.8    
3238	     	172.864  
3239	     	172.929  
3240	     	173      
3241	     	173.056  
3242	     	17

3589	     	191.439  
3590	     	191.495  
3591	     	191.539  
3592	     	191.587  
3593	     	191.627  
3594	     	191.683  
3595	     	191.735  
3596	     	191.783  
3597	     	191.831  
3598	     	191.875  
3599	     	191.923  
3600	     	191.971  
3601	     	192.019  
3602	     	192.063  
3603	     	192.116  
3604	     	192.16   
3605	     	192.204  
3606	     	192.257  
3607	     	192.301  
3608	     	192.351  
3609	     	192.399  
3610	     	192.449  
3611	     	192.497  
3612	     	192.549  
3613	     	192.597  
3614	     	192.641  
3615	     	192.697  
3616	     	192.741  
3617	     	192.789  
3618	     	192.841  
3619	     	192.889  
3620	     	192.937  
3621	     	192.981  
3622	     	193.025  
3623	     	193.073  
3624	     	193.123  
3625	     	193.171  
3626	     	193.217  
3627	     	193.258  
3628	     	193.31   
3629	     	193.354  
3630	     	193.402  
3631	     	193.45   
3632	     	193.498  
3633	     	193.539  
3634	     	193.591  
3635	     	193.638  
3636	     	19

3983	     	210.599  
3984	     	210.651  
3985	     	210.705  
3986	     	210.753  
3987	     	210.795  
3988	     	210.843  
3989	     	210.887  
3990	     	210.939  
3991	     	210.987  
3992	     	211.035  
3993	     	211.083  
3994	     	211.127  
3995	     	211.175  
3996	     	211.223  
3997	     	211.272  
3998	     	211.32   
3999	     	211.372  
4000	     	211.424  
4001	     	211.472  
4002	     	211.516  
4003	     	211.569  
4004	     	211.613  
4005	     	211.653  
4006	     	211.704  
4007	     	211.752  
4008	     	211.805  
4009	     	211.853  
4010	     	211.9    
4011	     	211.94   
4012	     	211.988  
4013	     	212.036  
4014	     	212.088  
4015	     	212.136  
4016	     	212.181  
4017	     	212.23   
4018	     	212.284  
4019	     	212.348  
4020	     	212.397  
4021	     	212.445  
4022	     	212.5    
4023	     	212.54   
4024	     	212.588  
4025	     	212.636  
4026	     	212.697  
4027	     	212.741  
4028	     	212.789  
4029	     	212.837  
4030	     	21

4377	     	230.183  
4378	     	230.228  
4379	     	230.273  
4380	     	230.315  
4381	     	230.356  
4382	     	230.414  
4383	     	230.46   
4384	     	230.506  
4385	     	230.559  
4386	     	230.621  
4387	     	230.673  
4388	     	230.73   
4389	     	230.775  
4390	     	230.819  
4391	     	230.863  
4392	     	230.906  
4393	     	230.947  
4394	     	230.989  
4395	     	231.036  
4396	     	231.079  
4397	     	231.123  
4398	     	231.171  
4399	     	231.215  
4400	     	231.255  
4401	     	231.301  
4402	     	231.349  
4403	     	231.391  
4404	     	231.434  
4405	     	231.484  
4406	     	231.533  
4407	     	231.586  
4408	     	231.628  
4409	     	231.689  
4410	     	231.732  
4411	     	231.774  
4412	     	231.819  
4413	     	231.867  
4414	     	231.914  
4415	     	231.959  
4416	     	232.003  
4417	     	232.052  
4418	     	232.104  
4419	     	232.153  
4420	     	232.196  
4421	     	232.242  
4422	     	232.285  
4423	     	232.331  
4424	     	23

4769	     	248.827  
4770	     	248.875  
4771	     	248.919  
4772	     	248.963  
4773	     	249.011  
4774	     	249.055  
4775	     	249.099  
4776	     	249.143  
4777	     	249.191  
4778	     	249.247  
4779	     	249.295  
4780	     	249.339  
4781	     	249.383  
4782	     	249.431  
4783	     	249.479  
4784	     	249.519  
4785	     	249.563  
4786	     	249.603  
4787	     	249.647  
4788	     	249.695  
4789	     	249.743  
4790	     	249.787  
4791	     	249.835  
4792	     	249.883  
4793	     	249.923  
4794	     	249.967  
4795	     	250.019  
4796	     	250.067  
4797	     	250.111  
4798	     	250.155  
4799	     	250.199  
4800	     	250.243  
4801	     	250.291  
4802	     	250.339  
4803	     	250.387  
4804	     	250.439  
4805	     	250.491  
4806	     	250.539  
4807	     	250.595  
4808	     	250.643  
4809	     	250.695  
4810	     	250.739  
4811	     	250.787  
4812	     	250.831  
4813	     	250.875  
4814	     	250.919  
4815	     	250.963  
4816	     	25

In [12]:
NOBJ = 3
NGEN = 5000
CXPB = 0.8
MUTPB = 0.2

P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))

MU = 92

ref_points = tools.uniform_reference_points(NOBJ, P)

IND_SIZE=100

In [13]:
for index in range(1, 11):  # Adjust the range as needed
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ) 
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("indices", random.sample, range(100), IND_SIZE)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", motsp) #Fitness Function

    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", mutInversion)
    toolbox.register("select", tools.selNSGA3, ref_points=ref_points)

    # Initialize statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("Tiempo", measure_time)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "Tiempo"
    
    print(f"Iteration {index}")
    start_time = time.time()
    res, logbook = nsga_iii(toolbox=toolbox,seed=index,stats=stats,logbook=logbook,NGEN=NGEN,MU=MU,CXPB=CXPB,MUTPB=MUTPB,verbose=True)

    fronts = tools.emo.sortLogNondominated(res, len(res))

    for i, inds in enumerate(fronts):
        par = [toolbox.evaluate(ind) for ind in inds]
        df = pd.DataFrame(par)

    df.to_csv(f'deap_fitness_nsga3_motsp_92_500_3_100-{index}.csv', index=False)
    log = pd.DataFrame(logbook)
    log.to_csv(f'deap_time_nsga3_motsp_92_500_3_100-{index}.csv', index=False)
    del(log, df, par, fronts, logbook, res, start_time)


C:\Users\Maria\miniconda3\envs\fs\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Maria\miniconda3\envs\fs\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Iteration 1
gen	evals	Tiempo  
1  	     	0.299987
2  	     	0.34396 
3  	     	0.392006
4  	     	0.416157
5  	     	0.440194
6  	     	0.468158
7  	     	0.492157
8  	     	0.516157
9  	     	0.544156
10 	     	0.572156
11 	     	0.596158
12 	     	0.636338
13 	     	0.656297
14 	     	0.684298
15 	     	0.708396
16 	     	0.732398
17 	     	0.840398
18 	     	0.900398
19 	     	0.927907
20 	     	0.955907
21 	     	0.979909
22 	     	1.00842 
23 	     	1.03242 
24 	     	1.05642 
25 	     	1.08442 
26 	     	1.10849 
27 	     	1.13253 
28 	     	1.15649 
29 	     	1.18049 
30 	     	1.20863 
31 	     	1.23267 
32 	     	1.27263 
33 	     	1.31263 
34 	     	1.36063 
35 	     	1.40479 
36 	     	1.44479 
37 	     	1.48079 
38 	     	1.5049  
39 	     	1.5289  
40 	     	1.5569  
41 	     	1.5849  
42 	     	1.61709 
43 	     	1.64509 
44 	     	1.66909 
45 	     	1.69709 
46 	     	1.72498 
47 	     	1.75298 
48 	     	1.78412 
49 	     	1.80666 
50 	     	1.83048 
51 	     	1.85452 


435	     	12.0253 
436	     	12.0483 
437	     	12.0723 
438	     	12.1043 
439	     	12.1295 
440	     	12.1534 
441	     	12.1814 
442	     	12.2055 
443	     	12.2333 
444	     	12.2573 
445	     	12.2813 
446	     	12.3093 
447	     	12.3375 
448	     	12.3655 
449	     	12.3895 
450	     	12.4175 
451	     	12.4413 
452	     	12.4653 
453	     	12.4893 
454	     	12.5255 
455	     	12.5495 
456	     	12.5695 
457	     	12.5975 
458	     	12.6218 
459	     	12.6458 
460	     	12.6698 
461	     	12.7018 
462	     	12.7283 
463	     	12.7603 
464	     	12.7883 
465	     	12.8083 
466	     	12.8337 
467	     	12.8657 
468	     	12.8857 
469	     	12.9337 
470	     	12.9737 
471	     	13.0017 
472	     	13.0298 
473	     	13.0498 
474	     	13.0778 
475	     	13.1098 
476	     	13.1323 
477	     	13.1563 
478	     	13.1803 
479	     	13.2043 
480	     	13.234  
481	     	13.258  
482	     	13.282  
483	     	13.31   
484	     	13.3341 
485	     	13.3621 
486	     	13.3861 
487	     	13

873	     	25.5338 
874	     	25.5658 
875	     	25.5978 
876	     	25.6338 
877	     	25.6658 
878	     	25.7018 
879	     	25.7298 
880	     	25.7618 
881	     	25.7938 
882	     	25.8258 
883	     	25.8618 
884	     	25.9058 
885	     	25.9418 
886	     	25.9698 
887	     	26.0138 
888	     	26.0418 
889	     	26.0738 
890	     	26.1058 
891	     	26.1338 
892	     	26.1698 
893	     	26.2018 
894	     	26.2298 
895	     	26.2618 
896	     	26.2898 
897	     	26.3298 
898	     	26.3578 
899	     	26.3858 
900	     	26.4218 
901	     	26.4498 
902	     	26.4858 
903	     	26.5178 
904	     	26.5498 
905	     	26.5778 
906	     	26.6138 
907	     	26.6418 
908	     	26.6818 
909	     	26.7138 
910	     	26.7418 
911	     	26.7859 
912	     	26.8138 
913	     	26.8618 
914	     	26.8939 
915	     	26.9338 
916	     	26.9658 
917	     	27.0058 
918	     	27.0378 
919	     	27.0738 
920	     	27.1018 
921	     	27.1338 
922	     	27.1658 
923	     	27.2018 
924	     	27.2338 
925	     	27

1290	     	39.8259 
1291	     	39.8502 
1292	     	39.8742 
1293	     	39.9022 
1294	     	39.9222 
1295	     	39.9502 
1296	     	39.9741 
1297	     	40.0061 
1298	     	40.0341 
1299	     	40.0581 
1300	     	40.0821 
1301	     	40.1061 
1302	     	40.1301 
1303	     	40.1541 
1304	     	40.1781 
1305	     	40.2061 
1306	     	40.2341 
1307	     	40.2621 
1308	     	40.2941 
1309	     	40.3221 
1310	     	40.3461 
1311	     	40.374  
1312	     	40.4021 
1313	     	40.43   
1314	     	40.45   
1315	     	40.474  
1316	     	40.498  
1317	     	40.526  
1318	     	40.554  
1319	     	40.578  
1320	     	40.602  
1321	     	40.63   
1322	     	40.654  
1323	     	40.6783 
1324	     	40.7023 
1325	     	40.7303 
1326	     	40.7503 
1327	     	40.7825 
1328	     	40.8065 
1329	     	40.8345 
1330	     	40.8665 
1331	     	40.8906 
1332	     	40.9145 
1333	     	40.9426 
1334	     	40.9666 
1335	     	40.9944 
1336	     	41.0264 
1337	     	41.0504 
1338	     	41.0745 
1339	     	41.0985 


...

3209	     	91.6368  
3210	     	91.671   
3211	     	91.6996  
3212	     	91.7301  
3213	     	91.7507  
3214	     	91.7773  
3215	     	91.8009  
3216	     	91.8303  
3217	     	91.8623  
3218	     	91.8881  
3219	     	91.9109  
3220	     	91.9397  
3221	     	91.9668  
3222	     	91.9931  
3223	     	92.0182  
3224	     	92.0429  
3225	     	92.0696  
3226	     	92.0941  
3227	     	92.1198  
3228	     	92.1508  
3229	     	92.1767  
3230	     	92.2019  
3231	     	92.2296  
3232	     	92.2592  
3233	     	92.2839  
3234	     	92.3142  
3235	     	92.3344  
3236	     	92.3619  
3237	     	92.395   
3238	     	92.4209  
3239	     	92.4501  
3240	     	92.4739  
3241	     	92.5036  
3242	     	92.5283  
3243	     	92.5499  
3244	     	92.5796  
3245	     	92.6086  
3246	     	92.636   
3247	     	92.6626  
3248	     	92.6841  
3249	     	92.7143  
3250	     	92.7429  
3251	     	92.7678  
3252	     	92.794   
3253	     	92.8156  
3254	     	92.84    
3255	     	92.8661  
3256	     	92

3600	     	102.676  
3601	     	102.701  
3602	     	102.734  
3603	     	102.762  
3604	     	102.798  
3605	     	102.82   
3606	     	102.856  
3607	     	102.895  
3608	     	102.929  
3609	     	102.972  
3610	     	103.01   
3611	     	103.051  
3612	     	103.09   
3613	     	103.11   
3614	     	103.141  
3615	     	103.172  
3616	     	103.198  
3617	     	103.22   
3618	     	103.244  
3619	     	103.269  
3620	     	103.293  
3621	     	103.319  
3622	     	103.343  
3623	     	103.37   
3624	     	103.401  
3625	     	103.43   
3626	     	103.451  
3627	     	103.479  
3628	     	103.5    
3629	     	103.53   
3630	     	103.554  
3631	     	103.581  
3632	     	103.611  
3633	     	103.643  
3634	     	103.665  
3635	     	103.7    
3636	     	103.727  
3637	     	103.751  
3638	     	103.773  
3639	     	103.802  
3640	     	103.839  
3641	     	103.869  
3642	     	103.894  
3643	     	103.922  
3644	     	103.949  
3645	     	103.976  
3646	     	104      
3647	     	10

3991	     	113.605  
3992	     	113.634  
3993	     	113.664  
3994	     	113.694  
3995	     	113.716  
3996	     	113.744  
3997	     	113.77   
3998	     	113.793  
3999	     	113.819  
4000	     	113.843  
4001	     	113.865  
4002	     	113.89   
4003	     	113.915  
4004	     	113.942  
4005	     	113.967  
4006	     	113.994  
4007	     	114.022  
4008	     	114.05   
4009	     	114.076  
4010	     	114.106  
4011	     	114.128  
4012	     	114.15   
4013	     	114.178  
4014	     	114.199  
4015	     	114.225  
4016	     	114.249  
4017	     	114.271  
4018	     	114.301  
4019	     	114.326  
4020	     	114.354  
4021	     	114.381  
4022	     	114.404  
4023	     	114.429  
4024	     	114.458  
4025	     	114.482  
4026	     	114.507  
4027	     	114.53   
4028	     	114.561  
4029	     	114.584  
4030	     	114.619  
4031	     	114.648  
4032	     	114.672  
4033	     	114.701  
4034	     	114.725  
4035	     	114.751  
4036	     	114.775  
4037	     	114.799  
4038	     	11

4388	     	125.095  
4389	     	125.13   
4390	     	125.151  
4391	     	125.18   
4392	     	125.2    
4393	     	125.23   
4394	     	125.252  
4395	     	125.297  
4396	     	125.343  
4397	     	125.383  
4398	     	125.422  
4399	     	125.463  
4400	     	125.499  
4401	     	125.53   
4402	     	125.568  
4403	     	125.611  
4404	     	125.644  
4405	     	125.67   
4406	     	125.7    
4407	     	125.729  
4408	     	125.751  
4409	     	125.78   
4410	     	125.82   
4411	     	125.851  
4412	     	125.883  
4413	     	125.917  
4414	     	125.951  
4415	     	125.986  
4416	     	126.019  
4417	     	126.053  
4418	     	126.095  
4419	     	126.128  
4420	     	126.155  
4421	     	126.181  
4422	     	126.218  
4423	     	126.249  
4424	     	126.277  
4425	     	126.309  
4426	     	126.335  
4427	     	126.359  
4428	     	126.384  
4429	     	126.41   
4430	     	126.435  
4431	     	126.461  
4432	     	126.493  
4433	     	126.528  
4434	     	126.551  
4435	     	12

4780	     	137.469  
4781	     	137.5    
4782	     	137.536  
4783	     	137.575  
4784	     	137.609  
4785	     	137.644  
4786	     	137.677  
4787	     	137.702  
4788	     	137.745  
4789	     	137.776  
4790	     	137.815  
4791	     	137.849  
4792	     	137.883  
4793	     	137.926  
4794	     	137.952  
4795	     	137.984  
4796	     	138.017  
4797	     	138.053  
4798	     	138.098  
4799	     	138.13   
4800	     	138.174  
4801	     	138.197  
4802	     	138.232  
4803	     	138.266  
4804	     	138.304  
4805	     	138.341  
4806	     	138.387  
4807	     	138.433  
4808	     	138.477  
4809	     	138.512  
4810	     	138.542  
4811	     	138.588  
4812	     	138.617  
4813	     	138.646  
4814	     	138.679  
4815	     	138.705  
4816	     	138.735  
4817	     	138.784  
4818	     	138.815  
4819	     	138.847  
4820	     	138.877  
4821	     	138.903  
4822	     	138.929  
4823	     	138.966  
4824	     	139.013  
4825	     	139.063  
4826	     	139.102  
4827	     	13